In [1]:
import h5py

/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [17]:
with h5py.File('../../scripts/data/killms_datapack_2.hdf5') as f, h5py.File('../../scripts/data/killms_datapack_3.hdf5') as g:
    g['/sol000/tec000/val'][...] = f['/sol000/tec000/val'][...]
    g['/sol000/tec000/weight'][...] = f['/sol000/tec000/weight'][...]

In [1]:
from bayes_tec.datapack import DataPack

In [3]:
with DataPack('../../scripts/data/ndppp_datapack.hdf5',readonly=True) as datapack:#'/home/albert/git/IonoTomo/src/ionotomo/data/NsolutionsDDE_2.5Jy_tecandphasePF_correctedlosoto_fulltime_dec27.h5',readonly=True) as datapack:
    tec, axes = datapack.tec
    patch_names,directions = datapack.get_sources(axes['dir'])
#     scalarphase,axes = datapack._solset.getSoltab('scalarphase000').getValues(reference='CS001HBA0',
#                                weight=False)
#     patch_names,directions = datapack.get_sources(axes['dir'])

2018-09-19 18:16:48,905 Appending to /net/lofar1/data1/albert/git/bayes_tec/scripts/data/ndppp_datapack.hdf5.
2018-09-19 18:16:48,912 Closing table.
2018-09-19 18:16:48,914 Reading from /net/lofar1/data1/albert/git/bayes_tec/scripts/data/ndppp_datapack.hdf5.
2018-09-19 18:16:49,132 Closing table.


In [3]:
print(len(patch_names))
import numpy as np
u, i = np.unique(patch_names,return_index=True)
print(len(u))

42
42


In [77]:
axes['dir']

array(['[Patch_0]', '[Patch_1]', '[Patch_10]', '[Patch_11]', '[Patch_12]',
       '[Patch_13]', '[Patch_14]', '[Patch_15]', '[Patch_16]',
       '[Patch_17]', '[Patch_18]', '[Patch_19]', '[Patch_20]',
       '[Patch_21]', '[Patch_22]', '[Patch_23]', '[Patch_24]',
       '[Patch_25]', '[Patch_26]', '[Patch_27]', '[Patch_28]',
       '[Patch_29]', '[Patch_3]', '[Patch_30]', '[Patch_31]',
       '[Patch_32]', '[Patch_33]', '[Patch_34]', '[Patch_35]',
       '[Patch_36]', '[Patch_37]', '[Patch_38]', '[Patch_39]',
       '[Patch_4]', '[Patch_40]', '[Patch_41]', '[Patch_42]',
       '[Patch_43]', '[Patch_6]', '[Patch_7]', '[Patch_8]', '[Patch_9]'],
      dtype='<U10')

In [78]:
list(patch_names).index(b'[Patch_20]')

1

In [5]:
directions[np.where(patch_names==b'[Patch_20]')]

<SkyCoord (ICRS): (ra, dec) in deg
    [(125.704414, 62.999622)]>

In [4]:
directions[np.where(patch_names==b'[Patch_6]')]

<SkyCoord (ICRS): (ra, dec) in deg
    [(127.4901, 64.64043)]>

In [81]:
tec[0,51,np.where(patch_names==b'[Patch_6]'),0]

IndexError: index 51 is out of bounds for axis 1 with size 42

In [82]:
tec.shape

(1, 42, 62, 3595)

In [83]:
axes['dir'][~np.isin(axes['dir'],patch_names)]

array(['[Patch_10]', '[Patch_30]', '[Patch_40]'], dtype='<U10')

In [4]:
with datapack:
    datapack.select(ant='RS*')
    axes = datapack.axes_phase
    print(axes['ant'])
    
    print(datapack.get_antennas(axes['ant']))

2018-09-19 18:16:52,960 Reading from /net/lofar1/data1/albert/git/bayes_tec/scripts/data/ndppp_datapack.hdf5.
['RS106HBA' 'RS205HBA' 'RS208HBA' 'RS210HBA' 'RS305HBA' 'RS306HBA'
 'RS307HBA' 'RS310HBA' 'RS406HBA' 'RS407HBA' 'RS409HBA' 'RS503HBA'
 'RS508HBA' 'RS509HBA']
(array([b'RS106HBA', b'RS205HBA', b'RS208HBA', b'RS210HBA', b'RS305HBA',
       b'RS306HBA', b'RS307HBA', b'RS310HBA', b'RS406HBA', b'RS407HBA',
       b'RS409HBA', b'RS503HBA', b'RS508HBA', b'RS509HBA'], dtype='|S9'), <SkyCoord (ITRS: obstime=J2000.000): (x, y, z) in m
    [(3829205.5, 469142.53, 5062181. ), (3831479.8, 463487.53, 5060990. ),
     (3847753.2, 466962.8 , 5048397. ), (3877827.5, 467536.6 , 5025445.5),
     (3828732.8, 454692.4 , 5063850.5), (3829771.2, 452761.7 , 5063243. ),
     (3837964.5, 449627.25, 5057357.5), (3845376.2, 413616.56, 5054796.5),
     (3818425. , 452020.28, 5071817.5), (3811649.5, 453459.9 , 5076729. ),
     (3824812.5, 426130.34, 5069252. ), (3824138.5, 459476.97, 5066858.5),
     (37971

In [120]:
sorter = np.argsort(axes['dir'])
dir_idx = np.searchsorted(axes['dir'], patch_names, sorter=sorter)
print(dir_idx)
print(patch_names[dir_idx])

[ 0 11  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 32 22
 23 24 25 26 27 28 29 30 31 37 33 34 35 36 38 39 40 41]
[b'[Patch_0]' b'[Patch_19]' b'[Patch_1]' b'[Patch_10]' b'[Patch_11]'
 b'[Patch_12]' b'[Patch_13]' b'[Patch_14]' b'[Patch_15]' b'[Patch_16]'
 b'[Patch_17]' b'[Patch_18]' b'[Patch_20]' b'[Patch_21]' b'[Patch_22]'
 b'[Patch_23]' b'[Patch_24]' b'[Patch_25]' b'[Patch_26]' b'[Patch_27]'
 b'[Patch_28]' b'[Patch_29]' b'[Patch_39]' b'[Patch_3]' b'[Patch_30]'
 b'[Patch_31]' b'[Patch_32]' b'[Patch_33]' b'[Patch_34]' b'[Patch_35]'
 b'[Patch_36]' b'[Patch_37]' b'[Patch_38]' b'[Patch_43]' b'[Patch_4]'
 b'[Patch_40]' b'[Patch_41]' b'[Patch_42]' b'[Patch_6]' b'[Patch_7]'
 b'[Patch_8]' b'[Patch_9]']


In [118]:
np.unique(patch_names[dir_idx]).shape

(42,)

In [95]:
with datapack:
    print(datapack.sources[1][[0,1]])

2018-09-19 16:03:16,088 Reading from /net/lofar1/data1/albert/git/bayes_tec/scripts/data/ndppp_datapack.hdf5.
[[2.2586963 1.138262 ]
 [2.261617  1.1390908]]
2018-09-19 16:03:16,097 Closing table.
